We're going to remake the same synthetic mixtures we ran through out Stan model.

Sadly our list of selected genes was not saved. So we have to rerun Stan on these mixtures.

Then we'll run them thru Cibersort.

Then we'll produce new figures with Cibersort results overlaid.

# Remake

Based on `2.1_oos_syntheticmixtures_markergenes.py` -- with significant changes

## marker genes + background genes selection (now reproducible)



In [2]:
import numpy as np
import matplotlib as mpl
import data
import models
import cache
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pystan
from time import time
from datetime import timedelta
import pickle
import dill

INFO:stancache.seed:Setting seed to 1245502385
INFO:root:Setting CACHE_DIR = /home/jovyan/modelcache/mz
INFO:stancache.seed:Setting seed to 1245502385


In [3]:
bindea = pd.read_csv('../curated_data/pure_samples/bindea/Bindea Immunome - Bindea paper.csv')
bindea.head()

,CellType,Symbol,AffymetrixID,UGCluster,EntrezGene,Name
0,aDC,CCL1,207533_at,Hs.72918,6346,chemokine (C-C motif) ligand 1
1,aDC,EBI3,219424_at,Hs.501452,10148,Epstein-Barr virus induced gene 3
2,aDC,INDO,210029_at,Hs.840,3620,"indoleamine-pyrrole 2,3 dioxygenase"
3,aDC,LAMP3,205569_at,Hs.518448,27074,lysosomal-associated membrane protein 3
4,aDC,OAS3,218400_at,Hs.528634,4940,"2'-5'-oligoadenylate synthetase 3, 100kDa"


In [4]:
marker_genes = bindea.Symbol.unique()
len(marker_genes)

579

In [5]:
model_name = 'model6.2'
by = 'SubSet'
total_num_genes = 800 # number we want to have total

In [6]:
sample_df = cache.cached(models.prep_sample_df,
                         sample_n=None)

INFO:stancache.stancache:prep_sample_df: cache_filename set to prep_sample_df.cached.sample_n_82429761595.pkl
INFO:stancache.stancache:prep_sample_df: Starting execution
INFO:stancache.stancache:prep_annotated_data: cache_filename set to prep_annotated_data.cached.default.pkl
INFO:stancache.stancache:prep_annotated_data: Starting execution
INFO:stancache.stancache:prep_annotated_data: Execution completed (0:12:57.256852 elapsed)
INFO:stancache.stancache:prep_annotated_data: Saving results to cache
INFO:stancache.stancache:prep_sample_df: Execution completed (0:13:12.366902 elapsed)
INFO:stancache.stancache:prep_sample_df: Saving results to cache


In [14]:
sample_df.SubSet.unique()

array(['CD4_Th2', 'CD8_Effector', 'B_CD5', 'B_Memory', 'CD4_Th1',
       'CD4_Naive', 'B_Naive', 'CD4_Effector_Memory', 'CD8_Central_Memory',
       'CD4_Central_Memory', 'CD4_Treg', 'CD8_Naive', 'CD4_Th17'], dtype=object)

In [15]:
all_genes = sample_df.gene_name.unique()
marker_genes_adjusted = list(set(marker_genes) - (set(marker_genes) - set(all_genes)))
genes_to_sample_from = list(set(all_genes) - set(marker_genes_adjusted))
print('desired size:', total_num_genes-len(marker_genes_adjusted))
sampled_genes = np.random.choice(genes_to_sample_from, size=total_num_genes-len(marker_genes_adjusted))
len(sampled_genes)

desired size: 281


281

In [26]:
selected_genes = np.concatenate([marker_genes_adjusted, sampled_genes]).tolist()
assert len(selected_genes) == total_num_genes

In [28]:
# pickle out selected gene list
import pickle
pickle.dump(selected_genes, open('logs/2.1.1_selectedgenes.pkl', 'wb'), pickle.HIGHEST_PROTOCOL)

In [29]:
filtered_sample_df = cache.cached(models.prep_sample_df,
                                 sample_gene_list=selected_genes)

INFO:stancache.stancache:prep_sample_df: cache_filename set to prep_sample_df.cached.sample_gene_list_14312972822.pkl
INFO:stancache.stancache:prep_sample_df: Starting execution
INFO:stancache.stancache:prep_annotated_data: cache_filename set to prep_annotated_data.cached.default.pkl
INFO:stancache.stancache:prep_annotated_data: Loading result from cache
INFO:stancache.stancache:prep_sample_df: Execution completed (0:00:05.361277 elapsed)
INFO:stancache.stancache:prep_sample_df: Saving results to cache


In [30]:
sample_df.shape, filtered_sample_df.shape

((2194416, 41), (50274, 41))

In [31]:
len(filtered_sample_df.gene_name.unique())

798

In [32]:
print('filtered sample df number of genes:', len(filtered_sample_df.gene_name.unique()))

filtered sample df number of genes: 798


In [33]:
pickle.dump(filtered_sample_df.gene_name.unique().tolist(),
            open('logs/2.1.1_selectedgenes.TRUE.pkl', 'wb'),
            pickle.HIGHEST_PROTOCOL)

In [34]:
training_df = filtered_sample_df

## synthetic mixtures

In [36]:
model_file = models.get_model_file(model_name=model_name)
stan_data = models.prep_stan_data(sample_df=training_df, test_df=None, by=by)
stan_data

{'C': 13,
 'G': 798,
 'M': 20,
 'N': 50274,
 'S': 63,
 'cell_features':                      CCR6+  CCR7+  CCR7-  CD127-  CD161+  CD19+  CD25+  CD27+  \
 SubSet                                                                          
 CD4_Th2                0.0    0.0    0.0     0.0     0.0    0.0    0.0    0.0   
 CD8_Effector           0.0    0.0    1.0     0.0     0.0    0.0    0.0    0.0   
 B_CD5                  0.0    0.0    0.0     0.0     0.0    1.0    0.0    0.0   
 B_Memory               0.0    0.0    0.0     0.0     0.0    1.0    0.0    1.0   
 CD4_Th1                0.0    0.0    0.0     0.0     0.0    0.0    0.0    0.0   
 CD4_Naive              0.0    1.0    0.0     0.0     0.0    0.0    0.0    0.0   
 B_Naive                0.0    0.0    0.0     0.0     0.0    1.0    0.0    0.0   
 CD4_Effector_Memory    0.0    0.0    1.0     0.0     0.0    0.0    0.0    0.0   
 CD8_Central_Memory     0.0    1.0    0.0     0.0     0.0    0.0    0.0    0.0   
 CD4_Central_Memory     0.0

In [35]:
def get_sample_ids_by_subset(training_df):
    return {subset: 
            sample_df[sample_df['SubSet'] == subset].new_sample_id.unique() for subset in sample_df.SubSet.unique()}
relevant_sample_ids = get_sample_ids_by_subset(training_df)
assert all([len(relevant_sample_ids[i]) > 0 for i in relevant_sample_ids])
relevant_sample_ids

{'B_CD5': array([ 3, 26, 31, 34]),
 'B_Memory': array([ 4, 29, 43, 48, 52]),
 'B_Naive': array([ 7, 21, 46, 54, 59]),
 'CD4_Central_Memory': array([17, 33, 49, 55, 61]),
 'CD4_Effector_Memory': array([10, 11, 23, 38, 51]),
 'CD4_Naive': array([ 6,  9, 15, 16, 45]),
 'CD4_Th1': array([ 5, 12, 19, 22, 57]),
 'CD4_Th17': array([30, 32, 39, 53, 60]),
 'CD4_Th2': array([ 1, 14, 35, 50, 63]),
 'CD4_Treg': array([18, 24, 36, 41, 44]),
 'CD8_Central_Memory': array([13, 28, 37, 42]),
 'CD8_Effector': array([ 2,  8, 20, 25, 56]),
 'CD8_Naive': array([27, 40, 47, 58, 62])}

In [39]:
def mix_cell_lines(sample_df, xdata, subsets, weights, sample_ids=None, new_sample_id=10001):
    """
    e.g. xdata=stan_data['x'], subsets=['B_Naive', 'B_Memory'], weights=[.5, .5], sample_ids=None
    if sample_ids are None, the first sample of each subset is used
    """
    
    assert len(weights) == len(subsets)
    if sample_ids:
        # confirm sample ids belong to the subsets we expect them to belong to.
        for subset, sample_id in zip(subsets, sample_ids):
            assert sample_id in relevant_sample_ids[subset]
    else:
        sample_ids = [relevant_sample_ids[subset][0] for subset in subsets]
    
    weights = np.array(weights)
    weights = weights / np.sum(weights) # normalize
    
    x2_data = pd.DataFrame(np.zeros((1, xdata.shape[1])), columns=xdata.columns)
    
    transformed_lines = []
    for subset, weight, sample_id in zip(subsets, weights, sample_ids):
        transformed = sample_df[sample_df['new_sample_id'] == sample_id].copy()
        transformed.loc[:,'est_counts'] *= weight
        transformed_lines.append(transformed)
        x2_data['SubSet[%s]' % subset] = weight
    
    mixed_sample = pd.concat(transformed_lines).groupby(         ['gene_name', 'new_gene_id'])['est_counts']        .sum().reset_index()
        
    mixed_sample['sample_id'] = new_sample_id
    
    return mixed_sample, x2_data

In [40]:
mix_cell_lines(sample_df=training_df, xdata=stan_data['x'], subsets=['B_Naive', 'B_Memory'], weights=[.5, .5], sample_ids=None)


(        gene_name  new_gene_id   est_counts  sample_id
 0           ABCB4            1   206.156191      10001
 1           ABCC4            2    96.394805      10001
 2           ABCG2            3     6.000000      10001
 3           ABHD2            4   556.193123      10001
 4            ABT1            5   359.500000      10001
 5      AC006483.5            6     0.000000      10001
 6      AC006486.9            7    26.421405      10001
 7      AC009414.1            8     0.000000      10001
 8      AC011295.1            9     0.863155      10001
 9      AC069282.6           10    67.500000      10001
 10    AC087380.14           11     2.500000      10001
 11     AC093166.2           12     0.000000      10001
 12     AC109815.2           13     0.000000      10001
 13     AC114814.2           14     0.000000      10001
 14     AC123900.2           15     0.000000      10001
 15          ACACB           16   284.955338      10001
 16          ACBD3           17   639.999975    

In [41]:
# here are the mixtures we want
mix1, mix1_x = mix_cell_lines(sample_df=training_df,
                              xdata=stan_data['x'],
                              subsets=['B_Naive', 'B_Memory'],
                              weights=[.5, .5],
                              sample_ids=[7, 4],
                              new_sample_id = 10001)

mix2, mix2_x = mix_cell_lines(sample_df=training_df,
                              xdata=stan_data['x'],
                              subsets=['B_Naive', 'B_Memory'],
                              weights=[.5, .5],
                              sample_ids=[21, 29],
                              new_sample_id = 10002)

mix3, mix3_x = mix_cell_lines(sample_df=training_df,
                              xdata=stan_data['x'],
                              subsets=['B_Naive', 'B_Memory'],
                              weights=[.25, .75],
                              sample_ids=[7, 4],
                              new_sample_id = 10003)

mix4, mix4_x = mix_cell_lines(sample_df=training_df,
                              xdata=stan_data['x'],
                              subsets=['B_Naive', 'B_Memory'],
                              weights=[.25, .75],
                              sample_ids=[21, 29],
                              new_sample_id = 10004)

mix5, mix5_x = mix_cell_lines(sample_df=training_df,
                              xdata=stan_data['x'],
                              subsets=['B_Naive', 'B_Memory'],
                              weights=[.75, .25],
                              sample_ids=[7, 4],
                              new_sample_id = 10005)

mix6, mix6_x = mix_cell_lines(sample_df=training_df,
                              xdata=stan_data['x'],
                              subsets=['B_Naive', 'B_Memory'],
                              weights=[.75, .25],
                              sample_ids=[21, 29],
                              new_sample_id = 10006)

# tregs vs naive B cells

mix7, mix7_x = mix_cell_lines(sample_df=training_df,
                              xdata=stan_data['x'],
                              subsets=['B_Naive', 'CD4_Treg'],
                              weights=[.5, .5],
                              sample_ids=[7, 18],
                              new_sample_id = 10007)

mix8, mix8_x = mix_cell_lines(sample_df=training_df,
                              xdata=stan_data['x'],
                              subsets=['B_Naive', 'CD4_Treg'],
                              weights=[.25, .75],
                              sample_ids=[7, 18],
                              new_sample_id = 10008)

mix9, mix9_x = mix_cell_lines(sample_df=training_df,
                              xdata=stan_data['x'],
                              subsets=['B_Naive', 'CD4_Treg'],
                              weights=[.5, .5],
                              sample_ids=[21, 24],
                              new_sample_id = 10009)

mix10, mix10_x = mix_cell_lines(sample_df=training_df,
                                xdata=stan_data['x'],
                              subsets=['B_Naive', 'CD4_Treg'],
                              weights=[.25, .75],
                              sample_ids=[21, 24],
                              new_sample_id = 10010)

In [42]:
# make a test_df and x2_data with all of them
test_df = pd.concat([mix1, mix2, mix3, mix4, mix5, mix6, mix7, mix8, mix9, mix10])
test_df['gene_id'] = test_df['new_gene_id']
x2_data = pd.concat([mix1_x,mix2_x,mix3_x,mix4_x,mix5_x,mix6_x,mix7_x,mix8_x,mix9_x,mix10_x])

In [53]:
for dat in [training_df, test_df]:
    dat.sort_values(['gene_id','sample_id'], inplace=True)
    dat['new_sample_cat'] = dat['sample_id'].astype('category')
    dat['new_sample_id'] = dat['new_sample_cat'].cat.codes+1

In [54]:
test_df.head()

,gene_name,new_gene_id,est_counts,sample_id,gene_id,new_sample_cat,new_sample_id
0,ABCB4,1,206.156191,10001,1,10001,1
0,ABCB4,1,759.406680,10002,1,10002,2
0,ABCB4,1,173.078081,10003,1,10003,3
0,ABCB4,1,444.020830,10004,1,10004,4
0,ABCB4,1,239.234300,10005,1,10005,5


In [55]:
x2_data.head()

,SubSet[B_CD5],SubSet[B_Memory],SubSet[B_Naive],SubSet[CD4_Central_Memory],SubSet[CD4_Effector_Memory],SubSet[CD4_Naive],SubSet[CD4_Th1],SubSet[CD4_Th17],SubSet[CD4_Th2],SubSet[CD4_Treg],SubSet[CD8_Central_Memory],SubSet[CD8_Effector],SubSet[CD8_Naive]
0,0.0,0.50,0.50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,0.0,0.50,0.50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,0.0,0.75,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,0.0,0.75,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,0.0,0.25,0.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# pickle out the synthetic mixtures -- so we can load into cibersort later

In [56]:
synthetic_mixtures = test_df.pivot(index='gene_name', columns='sample_id', values='est_counts')
synthetic_mixtures.head()

sample_id,10001,10002,10003,10004,10005,10006,10007,10008,10009,10010
gene_name,,,,,,,,,,
ABCB4,206.156191,759.406680,173.078081,444.020830,239.234300,1074.792531,138.156205,71.078102,695.589191,348.294596
ABCC4,96.394805,76.717060,131.041307,103.204185,61.748302,50.229935,67.409040,87.562660,67.534150,89.429820
ABCG2,6.000000,4.500000,6.000000,3.750000,6.000000,5.250000,23.499985,32.249977,13.499990,17.249985
ABHD2,556.193123,863.839295,602.581480,760.617562,509.804767,967.061028,934.681040,1170.313355,1127.120825,1155.539857
ABT1,359.500000,689.500000,434.750000,596.750000,284.250000,782.250000,344.500000,412.250000,719.000000,641.000000


In [57]:
synthetic_groundtruth = x2_data.transpose()
synthetic_groundtruth.columns = synthetic_mixtures.columns
synthetic_groundtruth.head()

sample_id,10001,10002,10003,10004,10005,10006,10007,10008,10009,10010
SubSet[B_CD5],0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00
SubSet[B_Memory],0.5,0.5,0.75,0.75,0.25,0.25,0.0,0.00,0.0,0.00
SubSet[B_Naive],0.5,0.5,0.25,0.25,0.75,0.75,0.5,0.25,0.5,0.25
SubSet[CD4_Central_Memory],0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00
SubSet[CD4_Effector_Memory],0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00


In [58]:
pickle.dump({'y_test': synthetic_mixtures, 'x_test': synthetic_groundtruth},
            open('logs/2.1.1_syntheticmixtures.pkl', 'wb'),
            pickle.HIGHEST_PROTOCOL)

synthetic_mixtures.to_csv('logs/2.1.1_syntheticmixtures.ytest.csv')
synthetic_groundtruth.to_csv('logs/2.1.1_syntheticmixtures.xtest.csv')

# prepare to run stan

In [59]:
test_data = {'N2': len(test_df.index),
             'S2': len(test_df.new_sample_id.unique()),
             'gene2': test_df.new_gene_id.values,
             'sample2': test_df.new_sample_id.values,
             'y2': test_df.est_counts.astype(int).values,
             'x2': x2_data, ## for easy access later
             }

stan_data.update(test_data)

In [60]:
stan_data['G']

798

In [62]:
pickle.dump(stan_data,
            open('logs/2.1.1_standata.pkl', 'wb'),
            pickle.HIGHEST_PROTOCOL)

# Run stan

To do so, we will actually run a python script saved out: `2.1.1_RunStan.py`.

To run interactively, we launch: `python -i 2.1.1_RunStan.py 2>&1 | tee logs/2.1.1_RunStan.py.consoleout.txt`

Do that from within a `docker exec -it maximz-dev-2 bash` shell. If you want, can `su jovyan` first.

# Retrain Cibersort on our raw samples

In [66]:
training_df.head()

,index,sample_id,filename,gene_name,est_counts,tpm,log1p_tpm,log1p_counts,CCR6+,CCR7+,...,log1p_tpm_rescaled_subset,log1p_tpm_rescaled,gene_cat,gene_id,B_cell,T_cell,new_gene_cat,new_gene_id,new_sample_cat,new_sample_id
0,0,1,ERR431566,ABCB4,1.000000,0.851882,0.616202,0.693147,0.0,0.0,...,1.998440,-0.188314,ABCB4,70,0,1,ABCB4,1,1,1
1,1,2,ERR431567,ABCB4,2.000000,0.397441,0.334642,1.098612,0.0,0.0,...,-0.115750,-0.636778,ABCB4,70,0,1,ABCB4,1,2,2
2,2,3,ERR431568,ABCB4,782.000170,28.925644,3.398716,6.663133,0.0,0.0,...,-6.838006,4.243620,ABCB4,70,1,0,ABCB4,1,3,3
3,3,4,ERR431569,ABCB4,139.999971,5.299960,1.840543,4.948760,0.0,0.0,...,-3.955303,1.761793,ABCB4,70,1,0,ABCB4,1,4,4
4,4,5,ERR431570,ABCB4,3.007070,0.197082,0.179887,1.388060,0.0,0.0,...,-2.390764,-0.883270,ABCB4,70,0,1,ABCB4,1,5,5


In [94]:
cibersort_reference = training_df.pivot(index='gene_name', columns='sample_id', values='est_counts')
cibersort_reference.head()

sample_id,1,2,3,4,5,6,7,8,9,10,...,54,55,56,57,58,59,60,61,62,63
gene_name,,,,,,,,,,,,,,,,,,,,,
ABCB4,1.000000,2.00000,782.00017,139.999971,3.00707,5.000000,272.31241,1.00000,14.03124,2.002061,...,396.427795,5.999999,8.999999,12.02080,4.00347,901.83768,4.00000,6.000000,7.000000,2.000000
ABCC4,174.714850,71.38846,11.24475,165.687810,59.50698,286.581810,27.10180,141.34336,177.54349,87.161680,...,12.107020,59.083843,72.030390,43.87995,87.84840,43.16080,54.91326,62.594610,162.481127,201.262660
ABCG2,43.000021,2.00000,5.00000,6.000000,6.00000,19.999975,6.00000,16.00000,44.99996,35.000000,...,2.000000,48.000000,21.999970,8.00000,26.00002,3.00000,36.00000,36.999966,18.000000,30.999997
ABHD2,870.505410,638.55064,612.95028,648.969836,819.47966,1413.857660,463.41641,790.15101,2553.48020,1082.887390,...,318.781630,1161.321060,1204.681780,903.92156,940.85575,470.75543,838.73480,1061.965090,625.161400,1200.626050
ABT1,276.000000,346.00000,644.00000,510.000000,324.00000,604.000000,209.00000,599.00000,1350.00000,461.000000,...,241.000000,483.000000,562.000000,301.00000,636.00000,358.00000,234.00000,453.000000,439.000000,349.000000


In [119]:
cibersort_reference.to_csv('2.1.1_cibersort_reference.txt', sep='\t')

In [120]:
!head -n 1 2.1.1_cibersort_reference.txt

gene_name	1	2	3	4	5	6	7	8	9	10	11	12	13	14	15	16	17	18	19	20	21	22	23	24	25	26	27	28	29	30	31	32	33	34	35	36	37	38	39	40	41	42	43	44	45	46	47	48	49	50	51	52	53	54	55	56	57	58	59	60	61	62	63


In [82]:
map_sampleid_to_subset = training_df.groupby('sample_id').SubSet.first()
map_sampleid_to_subset.head()

sample_id
1         CD4_Th2
2    CD8_Effector
3           B_CD5
4        B_Memory
5         CD4_Th1
Name: SubSet, dtype: object

In [121]:
map_sampleid_to_subset.to_csv('2.1.1_debug_sampleid_map_to_subset.txt', sep='\t')

In [87]:
presence_of_subset_in_sample = models.patsy_helper_nointercept(df=pd.DataFrame({'SubSet':map_sampleid_to_subset}), formula='SubSet').transpose()
presence_of_subset_in_sample.head()

sample_id,1,2,3,4,5,6,7,8,9,10,...,54,55,56,57,58,59,60,61,62,63
SubSet[B_CD5],0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SubSet[B_Memory],0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SubSet[B_Naive],0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
SubSet[CD4_Central_Memory],0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
SubSet[CD4_Effector_Memory],0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [89]:
# each subset is present in a few samples
presence_of_subset_in_sample.sum(axis=1)

SubSet[B_CD5]                  4.0
SubSet[B_Memory]               5.0
SubSet[B_Naive]                5.0
SubSet[CD4_Central_Memory]     5.0
SubSet[CD4_Effector_Memory]    5.0
SubSet[CD4_Naive]              5.0
SubSet[CD4_Th1]                5.0
SubSet[CD4_Th17]               5.0
SubSet[CD4_Th2]                5.0
SubSet[CD4_Treg]               5.0
SubSet[CD8_Central_Memory]     4.0
SubSet[CD8_Effector]           5.0
SubSet[CD8_Naive]              5.0
dtype: float64

In [91]:
# each sample only includes one subset
presence_of_subset_in_sample.sum(axis=0)

sample_id
1     1.0
2     1.0
3     1.0
4     1.0
5     1.0
6     1.0
7     1.0
8     1.0
9     1.0
10    1.0
11    1.0
12    1.0
13    1.0
14    1.0
15    1.0
16    1.0
17    1.0
18    1.0
19    1.0
20    1.0
21    1.0
22    1.0
23    1.0
24    1.0
25    1.0
26    1.0
27    1.0
28    1.0
29    1.0
30    1.0
     ... 
34    1.0
35    1.0
36    1.0
37    1.0
38    1.0
39    1.0
40    1.0
41    1.0
42    1.0
43    1.0
44    1.0
45    1.0
46    1.0
47    1.0
48    1.0
49    1.0
50    1.0
51    1.0
52    1.0
53    1.0
54    1.0
55    1.0
56    1.0
57    1.0
58    1.0
59    1.0
60    1.0
61    1.0
62    1.0
63    1.0
dtype: float64

In [116]:
# recode according to cibersort expected values: 1 is "present", 2 is "not present", 0 is "ignore"
cibersort_classes = presence_of_subset_in_sample.replace(0, 2).astype(int)
cibersort_classes = cibersort_classes.set_index(cibersort_classes.index.astype(str).str.replace(']', '').str.replace('[', '').str.replace('SubSet', ''))
cibersort_classes

sample_id,1,2,3,4,5,6,7,8,9,10,...,54,55,56,57,58,59,60,61,62,63
B_CD5,2,2,1,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
B_Memory,2,2,2,1,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
B_Naive,2,2,2,2,2,2,1,2,2,2,...,1,2,2,2,2,1,2,2,2,2
CD4_Central_Memory,2,2,2,2,2,2,2,2,2,2,...,2,1,2,2,2,2,2,1,2,2
CD4_Effector_Memory,2,2,2,2,2,2,2,2,2,1,...,2,2,2,2,2,2,2,2,2,2
CD4_Naive,2,2,2,2,2,1,2,2,1,2,...,2,2,2,2,2,2,2,2,2,2
CD4_Th1,2,2,2,2,1,2,2,2,2,2,...,2,2,2,1,2,2,2,2,2,2
CD4_Th17,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,1,2,2,2
CD4_Th2,1,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,1
CD4_Treg,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2


In [117]:
cibersort_classes.to_csv('2.1.1_cibersort_classes.txt', header=None, sep='\t')

In [118]:
!cat 2.1.1_cibersort_classes.txt

B_CD5	2	2	1	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	1	2	2	2	2	1	2	2	1	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2
B_Memory	2	2	2	1	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	1	2	2	2	2	2	2	2	2	2	2	2	2	2	1	2	2	2	2	1	2	2	2	1	2	2	2	2	2	2	2	2	2	2	2
B_Naive	2	2	2	2	2	2	1	2	2	2	2	2	2	2	2	2	2	2	2	2	1	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	1	2	2	2	2	2	2	2	1	2	2	2	2	1	2	2	2	2
CD4_Central_Memory	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	1	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	1	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	1	2	2	2	2	2	1	2	2	2	2	2	1	2	2
CD4_Effector_Memory	2	2	2	2	2	2	2	2	2	1	1	2	2	2	2	2	2	2	2	2	2	2	1	2	2	2	2	2	2	2	2	2	2	2	2	2	2	1	2	2	2	2	2	2	2	2	2	2	2	2	1	2	2	2	2	2	2	2	2	2	2	2	2
CD4_Naive	2	2	2	2	2	1	2	2	1	2	2	2	2	2	1	1	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	1	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2
CD4_Th1	2	2	2	2	1	2	2	2	2	2	2	1	2	2	2	2	2	2	1	2	2	1	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2	1	2	2	2	2	2	2
CD4_Th17	2	2	2	2	2	2	2	2	2	2	2	2	2	2	2

# Run Cibersort with LM22 and with our new signature matrix

See `Evaluating the deconvolution once we have a basis matrix.ipynb`

In [ ]:
# write out the reference profile mixtures
hard_mixtures_refprof = pd.DataFrame({'7 NoNoise':mix7, '8 SimpleNoise': mix8, '9 ComplexNoise': mix9})
hard_mixtures_refprof.index.name='GeneSymbol'
hard_mixtures_refprof.to_csv('test_mixtures/hard_reference_profile_mixtures.txt', sep='\t')
print(hard_mixtures_refprof.shape)
hard_mixtures_refprof.head()



In [ ]:
!java -Xmx3g -Xms3g -jar ~/CIBERSORT_package/CIBERSORT.jar -M test_mixtures/hard_reference_profile_mixtures.txt -B LM22.txt > test_mixtures/out/c3

In [ ]:
results3 = pd.read_csv('test_mixtures/out/c3', skiprows=list(range(6)), sep='\t')
results3.head()

In [ ]:
del results3['Unnamed: 26']
results3_dat = results3[results3.columns[1:-3]]
print(results3.shape, results3_dat.shape)
results3_dat.head()

In [ ]:
hard_ref_weights = pd.DataFrame(np.zeros(results3_dat.shape),columns=results3_dat.columns)
hard_ref_weights.loc[:,hard_cols] = weights
hard_ref_weights

In [ ]:
print('correlation between GT and estimate per mixture')
print(results3_dat.corrwith(hard_ref_weights, axis=1))